In [1]:
%matplotlib inline

In [1]:
import cv2
import os
import math
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten,MaxPooling2D,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
)
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from tensorflow.keras.optimizers import Adam,SGD
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.linear_model import LogisticRegression
from mostCommon import most_common,patient_score

In [2]:
IMAGE_WIDTH=224
IMAGE_HEIGHT=224

In [3]:
#Load all patients 
my_data = pd.read_csv('../data/Five_Frames_Per_patient.csv')
patients_data=pd.read_csv('../data/videos_data.csv')
data_array = my_data.to_numpy()
patients_array=patients_data["FileName"].to_numpy()
images_path = '../data/Five_Frames_Per_patient/'
no_of_test_patients=math.floor(patients_array.size*0.2) 
images_data = []
models = []

df=my_data
data_array=df.to_numpy()
for i in data_array:
    img = cv2.imread(images_path + i[0] + '.' + i[2], cv2.IMREAD_GRAYSCALE)
    img=  cv2.equalizeHist(img)
    img = cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT))
    img_2d = img.reshape(IMAGE_WIDTH ,IMAGE_HEIGHT,1)
    images_data.append(img_2d)

X=np.asarray(images_data)
y = np.asarray(my_data['Label'])
print(Counter(y))

Counter({'regular': 280, 'pneumonia': 180, 'covid': 150})


In [4]:
testPatient_indexes= [1,2,12,14,15,22,
                      33,34,37,42,43,52,53,
                      92,95,96,99,101,102,104,107,110,115,118]


In [5]:
#getting frame indexes of each patient
frames_index=[]
for i in testPatient_indexes:
  j=i*5
  z=j+5
  while j<z:
    frames_index.append(j)
    j+=1

# Adding the frames to X_test
X_test=np.take(X,frames_index,axis=0)
y_test=np.take(y,frames_index,axis=0)
# Removing the frames from X_train
X_train=np.delete(X,frames_index,axis=0)
y_train=np.delete(y,frames_index,axis=0)

# Scale the pixels
X_train=X_train/255.0
X_test=X_test/255.0

In [6]:
#Applying SMOTE
oversample = SMOTE()
X_train=np.resize(X_train,(X_train.shape[0],50176))
X_train,y_train=oversample.fit_resample(X_train, y_train)
counter=Counter(y_train)
print(counter)

Counter({'covid': 225, 'pneumonia': 225, 'regular': 225})


In [7]:
print('X_train: '+str(X_train.shape))
print('y_train: '+str(y_train.shape))
print('X_test: '+ str(X_test.shape))
print('y_test: '+ str(y_test.shape))

X_train: (675, 50176)
y_train: (675,)
X_test: (120, 224, 224, 1)
y_test: (120,)


In [8]:
model=LogisticRegression(random_state=42,solver='lbfgs',max_iter=10000)
model.fit(X_train,y_train)

LogisticRegression(max_iter=10000, random_state=42)

In [9]:
X_test=np.resize(X_test,(X_test.shape[0],50176))
y_pred=model.predict(X_test)

In [10]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred, digits=3))
patient_score(y_test,y_pred)

[[21  0  9]
 [ 0 24 11]
 [ 0  5 50]]
              precision    recall  f1-score   support

       covid      1.000     0.700     0.824        30
   pneumonia      0.828     0.686     0.750        35
     regular      0.714     0.909     0.800        55

    accuracy                          0.792       120
   macro avg      0.847     0.765     0.791       120
weighted avg      0.819     0.792     0.791       120

[[ 4  0  2]
 [ 0  5  2]
 [ 0  1 10]]
              precision    recall  f1-score   support

       covid      1.000     0.667     0.800         6
   pneumonia      0.833     0.714     0.769         7
     regular      0.714     0.909     0.800        11

    accuracy                          0.792        24
   macro avg      0.849     0.763     0.790        24
weighted avg      0.820     0.792     0.791        24



In [11]:
from sklearn.model_selection import cross_val_score

scores=cross_val_score(model, X_train, y_train, cv=5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.87 accuracy with a standard deviation of 0.08


In [12]:


IMAGE_WIDTH=224
IMAGE_HEIGHT=224
#Load all patients 
my_data = pd.read_csv('../data/Five_Frames_Per_patient.csv')
patients_data=pd.read_csv('../data/videos_data.csv')
data_array = my_data.to_numpy()
patients_array=patients_data["FileName"].to_numpy()
images_path = '../data/Five_Frames_Per_patient/'
no_of_test_patients=math.floor(patients_array.size*0.2) 
images_data = []
models = []

df=my_data
data_array=df.to_numpy()
for i in data_array:
    img = cv2.imread(images_path + i[0] + '.' + i[2], cv2.IMREAD_GRAYSCALE)
    img=  cv2.equalizeHist(img)
    img = cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT))
    img_2d = img.reshape(IMAGE_WIDTH ,IMAGE_HEIGHT,1)
    images_data.append(img_2d)

X=np.asarray(images_data)
y = np.asarray(my_data['Label'])
print(Counter(y))

testPatient_indexes= [1,2,12,14,15,22,
                      33,34,37,42,43,52,53,
                      92,95,96,99,101,102,104,107,110,115,118]
#getting frame indexes of each patient
frames_index=[]
for i in testPatient_indexes:
  j=i*5
  z=j+5
  while j<z:
    frames_index.append(j)
    j+=1

# Adding the frames to X_test
X_test=np.take(X,frames_index,axis=0)
y_test=np.take(y,frames_index,axis=0)
# Removing the frames from X_train
X_train=np.delete(X,frames_index,axis=0)
y_train=np.delete(y,frames_index,axis=0)

# Scale the pixels
X_train=X_train/255.0
X_test=X_test/255.0
X_train=np.resize(X_train,(X_train.shape[0],50176))
print('X_train: '+str(X_train.shape))
print('y_train: '+str(y_train.shape))
print('X_test: '+ str(X_test.shape))
print('y_test: '+ str(y_test.shape))
model=LogisticRegression(random_state=42,solver='lbfgs',max_iter=10000)
model.fit(X_train,y_train)
X_test=np.resize(X_test,(X_test.shape[0],50176))
y_pred=model.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred, digits=3))
patient_score(y_test,y_pred)
from sklearn.model_selection import cross_val_score

scores=cross_val_score(model, X_train, y_train, cv=5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Counter({'regular': 280, 'pneumonia': 180, 'covid': 150})
X_train: (490, 50176)
y_train: (490,)
X_test: (120, 224, 224, 1)
y_test: (120,)
[[21  0  9]
 [ 0 22 13]
 [ 0  5 50]]
              precision    recall  f1-score   support

       covid      1.000     0.700     0.824        30
   pneumonia      0.815     0.629     0.710        35
     regular      0.694     0.909     0.787        55

    accuracy                          0.775       120
   macro avg      0.836     0.746     0.774       120
weighted avg      0.806     0.775     0.774       120

[[ 4  0  2]
 [ 0  5  2]
 [ 0  1 10]]
              precision    recall  f1-score   support

       covid      1.000     0.667     0.800         6
   pneumonia      0.833     0.714     0.769         7
     regular      0.714     0.909     0.800        11

    accuracy                          0.792        24
   macro avg      0.849     0.763     0.790        24
weighted avg      0.820     0.792     0.791        24

0.64 accuracy with a stand

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
IMAGE_WIDTH=224
IMAGE_HEIGHT=224
#Load all patients 
my_data = pd.read_csv('../data/Five_Frames_Per_patient.csv')
patients_data=pd.read_csv('../data/videos_data.csv')
data_array = my_data.to_numpy()
patients_array=patients_data["FileName"].to_numpy()
images_path = '../data/Five_Frames_Per_patient/'
no_of_test_patients=math.floor(patients_array.size*0.2) 
images_data = []
models = []

df=my_data
data_array=df.to_numpy()
for i in data_array:
    img = cv2.imread(images_path + i[0] + '.' + i[2], cv2.IMREAD_GRAYSCALE)
    img=  cv2.equalizeHist(img)
    img = cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT))
    img_1d = img.reshape(224 * 224)
    images_data.append(img_1d)

X=np.asarray(images_data)
y = np.asarray(my_data['Label'])
print(Counter(y))
scaler = StandardScaler()
# Fit on training set only
scaler.fit(X)
# Apply transform on both training and test set

X = scaler.transform(X)
pca = PCA(.95)
pca.fit(X)
X = pca.transform(X)

testPatient_indexes= [1,2,12,14,15,22,
                      33,34,37,42,43,52,53,
                      92,95,96,99,101,102,104,107,110,115,118]
#getting frame indexes of each patient
frames_index=[]
for i in testPatient_indexes:
  j=i*5
  z=j+5
  while j<z:
    frames_index.append(j)
    j+=1

# Adding the frames to X_test
X_test=np.take(X,frames_index,axis=0)
y_test=np.take(y,frames_index,axis=0)
# Removing the frames from X_train
X_train=np.delete(X,frames_index,axis=0)
y_train=np.delete(y,frames_index,axis=0)


print('X_train: '+str(X_train.shape))
print('y_train: '+str(y_train.shape))
print('X_test: '+ str(X_test.shape))
print('y_test: '+ str(y_test.shape))
model=LogisticRegression(random_state=42,solver='lbfgs',max_iter=10000)
model.fit(X_train,y_train)

y_pred=model.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred, digits=3))
patient_score(y_test,y_pred)
from sklearn.model_selection import cross_val_score

scores=cross_val_score(model, X_train, y_train, cv=5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Counter({'regular': 280, 'pneumonia': 180, 'covid': 150})
X_train: (490, 99)
y_train: (490,)
X_test: (120, 99)
y_test: (120,)
[[19  2  9]
 [ 0 18 17]
 [ 0  3 52]]
              precision    recall  f1-score   support

       covid      1.000     0.633     0.776        30
   pneumonia      0.783     0.514     0.621        35
     regular      0.667     0.945     0.782        55

    accuracy                          0.742       120
   macro avg      0.816     0.698     0.726       120
weighted avg      0.784     0.742     0.733       120

[[ 4  0  2]
 [ 0  4  3]
 [ 0  0 11]]
              precision    recall  f1-score   support

       covid      1.000     0.667     0.800         6
   pneumonia      1.000     0.571     0.727         7
     regular      0.688     1.000     0.815        11

    accuracy                          0.792        24
   macro avg      0.896     0.746     0.781        24
weighted avg      0.857     0.792     0.786        24

0.63 accuracy with a standard deviatio